In [1]:
import os
import numpy as np
import pickle
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
skeleton_path = 'skeleton-reality-data/re_skeleton_raw_1.txt'
with open(skeleton_path, 'r') as f:
    skeleton_list = f.readlines()

In [3]:
ACTION_NAMES = ['Wave','Hammer','Smash','Catch','Forward Punch','Throw','Draw X','Draw Tick','Draw Circle',
               'Clapping Hand','Two hand Wave','Side Boxing','Bend','Forward Kick','Side Kick','Jogging',
               'Tennis Swing','Tennis Serve','Golf Swing','Pickup&throw']

In [4]:
# From frame 30th to 160th
action_11 = skeleton_list[610:775]

In [5]:
len(action_11)

165

In [6]:
action_11[0]

'0.186174 -0.236540 3.022136 0.191174 -0.184076 3.079624 0.210051 0.146475 3.095182 0.207977 0.314886 3.115148 0.042236 0.117579 3.093050 -0.121019 0.262842 3.056628 -0.248450 0.414940 3.006071 -0.299031 0.495962 2.976345 0.359864 0.025460 3.070377 0.406240 -0.224912 3.018939 0.389909 -0.419009 2.931433 0.369122 -0.508232 2.898142 0.108877 -0.300519 3.005601 0.069882 -0.730583 2.953170 0.042826 -1.062970 2.918303 0.013056 -1.126038 2.863394 0.254631 -0.314573 3.003804 0.288527 -0.728296 2.961001 0.305925 -1.074903 2.941937 0.312165 -1.118699 2.892645 -0.006577 0.991491 0.130009 0.764766 \n'

In [7]:
for i in range(len(action_11)):
    action_11[i] = action_11[i].rstrip('\n')

In [8]:
action_11[0]

'0.186174 -0.236540 3.022136 0.191174 -0.184076 3.079624 0.210051 0.146475 3.095182 0.207977 0.314886 3.115148 0.042236 0.117579 3.093050 -0.121019 0.262842 3.056628 -0.248450 0.414940 3.006071 -0.299031 0.495962 2.976345 0.359864 0.025460 3.070377 0.406240 -0.224912 3.018939 0.389909 -0.419009 2.931433 0.369122 -0.508232 2.898142 0.108877 -0.300519 3.005601 0.069882 -0.730583 2.953170 0.042826 -1.062970 2.918303 0.013056 -1.126038 2.863394 0.254631 -0.314573 3.003804 0.288527 -0.728296 2.961001 0.305925 -1.074903 2.941937 0.312165 -1.118699 2.892645 -0.006577 0.991491 0.130009 0.764766 '

In [9]:
for i in range(len(action_11)):
    action_11[i] = action_11[i].split(' ')[:-1]

In [10]:
for i in range(len(action_11)):
    action_11[i] = [float(coor) for coor in action_11[i]]
    action_11[i] = action_11[i][:60]

In [11]:
Ns4_11 = np.array([9,11,13,20])
Ns4_11 = Ns4_11-1

In [12]:
x,y,z = [], [], []
for i in range(len(action_11)):
    for j in range(len(action_11[i])):
        if j%3 == 0:
            x.append(action_11[i][j])
        elif j%3 == 1:
            y.append(action_11[i][j])
        elif j%3 == 2:
            z.append(action_11[i][j])

In [13]:
x = np.asarray(x)
y = np.asarray(y)
z = np.asarray(z)

In [14]:
x.shape

(3300,)

In [15]:
def reshape_skeleton(X):
    noFrames = int(X.shape[0]/20)
    reshaped_X = []
    for i in range(20):
        reshaped_X.append([X[j] for j in range(i, X.shape[0], 20)])
    reshaped_X = np.asarray(reshaped_X)
    return reshaped_X

In [16]:
X = reshape_skeleton(x)
Y = reshape_skeleton(y)
Z = reshape_skeleton(z)

In [17]:
X = X[Ns4_11]

In [18]:
Y = Y[Ns4_11]
Z = Z[Ns4_11]

In [19]:
from norm import *
from MostJoints.getMostInformativeJoints import *
from utils import *

In [20]:
def calculateCovarianceMat(X, Y, Z, T, nLevels, overlap = False, timeVar=True):
    nFrames = X.shape[0]
    nJoins = X.shape[1]
    assert Y.shape[0] == nFrames
    assert Z.shape[0] == nFrames
    assert T.shape[0] == nFrames
    assert Y.shape[1] == nJoins
    assert Z.shape[1] == nJoins

    # Normalize skeleton coordinators

    normX = normCord(X)
    normY = normCord(Y)
    normZ = normCord(Z)
    normT = normSeT(T)

    # Create a list of full covariance matrices
    fullCovMats = [[] for i in range(nLevels)]
    covMats = [[] for i in range(nLevels)]

    if timeVar:
        sizeMatrix = nJoins*3+1
    else:
        sizeMatrix = nJoins*3

    listIdxMatrix = getValueMatrix(sizeMatrix) # get half of covariance matrix indexes


    for l in range(1,nLevels+1):
        # Compute covariance matrixes for each level
        nofMats = 2**(l-1)
        sizeWindow = 1/nofMats
        stepWindow = sizeWindow
        if overlap:
            stepWindow = stepWindow/2
            nofMats = nofMats*2-1
        startFrameTimes = [stepWindow*i for i in range(nofMats)]
        fullCovMats[l-1] = [[] for i in range(nofMats)]
        covMats[l-1] = [[] for i in range(nofMats)]
        for i in range(len(startFrameTimes)):
            startTime = startFrameTimes[i]
            endTime = startFrameTimes[i] + sizeWindow + 2*np.finfo(float).eps
            sliceInds = [j for j in range(T.shape[0]) if normT[j] >= startTime and normT[j] < endTime]
            sliceX = normX[sliceInds, :]
            sliceY = normY[sliceInds, :]
            sliceZ = normZ[sliceInds, :]
            sliceT = normT[sliceInds]
            if not timeVar:
                sliceVars = np.concatenate((np.concatenate((sliceX,sliceY), axis=1), sliceZ), axis=1)
            else:
                sliceVars = np.concatenate((np.concatenate((sliceX, sliceY), axis=1), np.concatenate((sliceZ, sliceT), axis=1)), axis=1)
            covarianceMat = np.cov(sliceVars.T)
            fullCovMats[l-1][i] = covarianceMat
            # Get half of covarianceMat and save it as a vector (1-D matrix)
            one_half_vector = []
            mask = np.zeros_like(covarianceMat, dtype=np.bool)
            mask[np.triu_indices_from(mask)] = True
            for row in range(covarianceMat.shape[0]):
                for column in range(covarianceMat.shape[0]):
                    if mask[row][column] == True:
                        one_half_vector.append(covarianceMat[row][column])
            covMats[l-1][i] = np.asarray(one_half_vector)
        covMats[l-1] = np.asarray(covMats[l-1])
        
    covMats = np.asarray(covMats)
    vec = np.empty(0)
    for i in range(covMats.shape[0]):
        for j in range(covMats[i].shape[0]):
            vec = np.hstack((vec, covMats[i][j]))
    return fullCovMats, vec

def get_covariance_vector(x,y,z,t):
    fullCovmat, vec_covMat = calculateCovarianceMat(x.T, y.T, z.T, t, nLevels=3, overlap=True, timeVar=False)
    return vec_covMat

def make_preprocessed_training_data():
    T = np.arange(1, X.shape[1]+1).T
    vec_covMat = get_covariance_vector(X,Y,Z,T)
    return vec_covMat

In [21]:
pickle_in = open('models/as2_svm_model.pkl', 'rb')
model = pickle.load(pickle_in)

In [22]:
vec_covMat = make_preprocessed_training_data()

In [23]:
X_test = []
X_test.append(vec_covMat)
X_test = np.asarray(X_test)

In [24]:
y_pred = model.predict(X_test)
print(y_pred[0])
if y_pred[0] == 14:
    y_pred[0] == 1
elif y_pred[0] == 13:
    y_pred[0] == 2
# print("Acting : ", ACTION_NAMES[y_pred[0]-1])

14
